# CrewAI와 Tool 결합

In [1]:
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

## 라이브러리 임포트
### crewai_tools
 - SerperDevTool : Serper api를 활용한 결과물을 받는 도구, 구글 웹검색 도구
 - WebsiteSearchTool : 특정 키워드와 특정 웹사이트를 주면, 해당 웹 사이트에서 키워드와 관련이 있는 부분만 가져옴.
 - ScrapeWebsiteTool : 어떤 URL을 주면 해당 URL이 있는 텍스트를 스크태핑해서 가져오는 도구

In [3]:
from crewai import Agent, Task, Crew
from crewai.process import Process
from crewai_tools import SerperDevTool, WebsiteSearchTool, ScrapeWebsiteTool
from langchain_openai import ChatOpenAI

## 필요한 객체 생성

In [4]:
llm = ChatOpenAI(model="gpt-4o-mini")

search_tool = SerperDevTool()
web_rag_tool = WebsiteSearchTool()
scrap_tool = ScrapeWebsiteTool()

c:\Users\Admin\miniconda3\envs\agent_env\Lib\site-packages\alembic\config.py:577: DeprecationWarning: No path_separator found in configuration; falling back to legacy splitting on spaces, commas, and colons for prepend_sys_path.  Consider adding path_separator=os to Alembic config.
  util.warn_deprecated(


## Agent 정의

In [5]:
researcher = Agent(
    role = "테크 트렌드 연구원",
    goal="인공지능 분야의 최신 기술 트렌드를 한국어로 제공합니다. 지금은 2025년 6월입니다.",
    backstory="기술 트렌드에 예리한 안목을 지닌 전문 분석가이자 AI 개발자입니다.",
    tools=[search_tool, web_rag_tool],
    verbose=True,
    max_iter=5,
    llm=llm
)

#### allow_delegatoin = False
 - 해당 agent가 task를 수행할 때 혼자 힘으로 하기 힘든 경우 다른 agent에 위임하는 설정
 - default = True
 - False로 하지 않으면 수행중에 researcher에게 작업을 넘기는 경우가 발생하는데 이 것을 발생하지 않도록 제한하기 위함

In [6]:
writer = Agent(
    role="뉴스레터 작성자",
    goal="최신 AI 기술 트렌드에 대한 매력적인 테크 뉴스레터를 한국어로 작성하세요. 지금은 2025년 6월입니다.",
    backstory="기술에 대한 열정을 가진 숙련된 작가입니다.",
    verbose=True,
    allow_delegation=False,  
    llm=llm
)

## Task 정의

In [7]:
research = Task(
    description="AI 업계의 최신 기술 동향을 조사하고 요약을 제공하세요.",
    expected_output="AI 업계에서 가장 주목받는 3대 기술 동향과 그 중요성에 대한 신선한 관점을 요약한 글",
    agent=researcher
)

In [8]:
write = Task(
    description="""
테크 트렌드 연구원의 요약을 바탕으로 AI 산업에 대한 매력적인 테크 뉴스레터를 작성하세요.
테크 뉴스레터이므로 전문적인 용어를 사용해도 괜찮습니다.
""",
    expected_output="최신 기술 관련 소식을 재밌는 말투로 소개하는 4문단짜리 마크다운 형식 뉴스레터",
    agent=writer,
    output_file=r"./new_post.md"
)

## Crew 정의

In [13]:
crew = Crew(
    agents=[researcher, writer],
    tasks=[research, write],
    verbose=False,
    process=Process.sequential,
    output_log_file="logs/crew_run.json",
)

## Crew 실행

In [ ]:
result = crew.kickoff()
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 4ec755ba-4301-41f4-bccc-3443250440fb                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: 4ec755ba-4301-41f4-bccc-3443250440fb                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

ValueError: Failed to log message: [Errno 2] No such file or directory: 'logs/crew_run.json'

## 마크다운으로 표시

In [ ]:
from IPython.display import display, Markdown
display(Markdown(result.raw))